## Lab 10d: Optimize the feature set of a rack-mounted M2 MacPro Server
The [new MacPro with the M2 Ultra chip has been released](https://www.apple.com/shop/buy-mac/mac-pro/rack#). The MacPro M2 has several configuration options broadly organized into five categories: `{CPU, Memory, Storage, Accessories, Software}` with multiple options per category:
* The `CPU` category has `2` options. Only one option can be selected from the `CPU` category
* The `Memory` category has `3` options. Only one option can be selected from the `Memory` category.
* The `Storage` category has `4` options. Only one option can be selected from the `Storage` category.
* The `Accessories` category has `3` options. Only one option can be selected from the `Accessories` category
* The `Software` category has `2` options. Neither or both options can be selected from the `Software` category

### Problem statement
Using a linear utility model with budget and feature constraints, determine the optimal features for a rack-mounted MacPro M2. This problem resembles lecture examples, except that our decision variables will be binary: $x_{i}\in{0,1}$, where `0` means not selecting feature $i`, and `1` means selecting it. 

Formally, an agent has a set of $n$ configuration options $X = \left\{x_{i}\right\}_{i=1}^{n}$, a `Linear` utility function, and a total of $I$ units of resource to allocate, e.g., money, and potentially other constraints. An optimal agent maximizes its utility subject to its resource and other constraints:
$$
\begin{eqnarray}
\text{maximize}~\mathcal{O} &=& \sum_{i\in{1,\dotsc,n}}\alpha_{i}x_{i} \\
\text{subject to}~\sum_{i\in{1,\dotsc,n}}c_{i}x_{i} & = & I\\
\text{and}~\mathbf{C}\mathbf{x} & \leq & \mathbf{b} \\
\text{and}~x_{i}&\in&{0,1}\qquad{i=1,2,\dots,n}
\end{eqnarray}
$$
The quantity $c_{i}\geq{0}~\forall{i}$ denotes the cost of option $i$, $\alpha_{i}$ denotes user-specified coefficient in the `Linear` utility function, $x_{i}\in{0,1}$ represents the choice of option $i$, and $\mathbf{C}\mathbf{x} \leq \mathbf{b}$ represents additional constraints governing the decision.

#### List of Tasks
* __Task 1__: Specify $\alpha$- and $c$-vectors for the problem
* __Task 2__: Specify the additional constraint matrix $\mathbf{C}$
* __Task 3__: Specify the problem object and solve the problem
* __Task 4__: Try at least two different weighting schemes and different budget values, and explore how these design choices influence the optimal configuration (one of these __can__ be the `default` values specified below).

## Setup
We set up the computational environment by including [the `Include. jl` file](Include.jl) using [the `include(...)` method](https://docs.julialang.org/en/v1/base/base/#Base.include). The [`Include.jl` file](Include.jl) loads external packages and functions we will use in these examples. 
* For additional information on functions and types used in this example, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-5760-PS1-OptimalMacProRackDesign-Fall-2023`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
   Installed Glib_jll ─ v2.76.5+0
  No Changes to `~/Desktop/julia_work/CHEME-5760-PS1-OptimalMacProRackDesign-Fall-2023/Project.toml`
    Updating `~/Desktop/julia_work/CHEME-5760-PS1-OptimalMacProRackDesign-Fall-2023/Manifest.toml`
  [7746bdde] ↑ Glib_jll v2.74.0+2 ⇒ v2.76.5+0
Precompiling project...
  ✓ Glib_jll
  ✓ Cairo_jll
  ✓ Qt6Base_jll
  ✓ HarfBuzz_jll
  ✓ libass_jll
  ✓ FFMPEG_jll
  ✓ FFMPEG
  ✓ GR_jll
  ✓ GR
  ✓ Plots
  ✓ Plots → UnitfulExt
  ✓ VLDecisionsPackage
  12 dependencies successfully precompiled in 47 seconds. 178 already precompiled.


#### Specify constants and other static stuff

In [2]:
number_of_choices = 14;
bounds_array = Array{Float64,2}(undef, number_of_choices,2)
for i ∈ 1:number_of_choices
    bounds_array[i,1] = 0.0
    bounds_array[i,2] = 1.0;
end

## Task 1: Specify the configuration array
In this task, you'll specify the `configuration_array` a `14` $\times$ `2` array holding perception and cost information about the problem. Each row of the `configuration_array` contains data for a particular MacPro configuration option. The first column contains the coefficients of the `Linear` utility function, i.e., the elements of the $\alpha$-vector, while the unit price of the features, i.e., elements of the $c$-vector, are in the second column.  

The prices of each configuration option have been estimated from the [Apple MacPro server website](https://www.apple.com/shop/buy-mac/mac-pro/rack#). We'll use a category-based weighting scheme. In each of the five categories, allocate `1` unit of weight:
* In each category, the coefficients in the `Linear` utility function for options in that category must sum to one.
For example, if you have equal feelings about three options in a category, then `0.33, 0.33, 0.33` would be a typical scheme. On the other hand, if you are excited about feature `1` over the other two in the category, then `0.8,0.1,0.1` could be an appropriate weight scheme.

In [3]:
configuration_array = [

    # CPU options
    0.5 2640.0    ; # 1 CPU 1
    0.5 3649.0    ; # 2 CPU 2

    # Memory options
    0.333 3840.0  ; # 3 Memory 1
    0.333 4640.0  ; # 4 Memory 2
    0.333 3600.0  ; # 5 Memory 3

    # Storage options -
    0.25 1440.0   ; # 6 Storage 1
    0.25 1840.0   ; # 7 Storage 2
    0.25 2440.0   ; # 8 Storage 3
    0.25 3640.0   ; # 9 Storage 4

    # Accessory options
    0.333 79.0    ; # 10 Accessory 1
    0.333 129.0   ; # 11 Accessory 2
    0.333 149.0   ; # 12 Accessory 3
    
    # Software options
    0.5 299.0     ; # 13 Software 1
    0.5 149.0     ; # 14 Software 2
];

### Task 2: Specify the feature constraint matrix $\mathbf{C}$
In this task, we'll set up the constraint matrix $\mathbf{C}$. In each category, only a finite number of options can be selected simultaneously, typically only a single option, with the exception being the `Software` category, which is unconstrained (can have from zero up to two items selected). Because the decision variables are binary, we can implement this requirement with an additional set of constraints of the form:
$$
\begin{equation}
\mathbf{C}\cdot\mathbf{x} = \mathbf{1}
\end{equation}
$$
where $\mathbf{C}$ denotes the constraint matrix, $\mathbf{x}$ denotes the choice vector and $\mathbf{1}$ denotes a vector of `1`'s. Specify the $\mathbf{C}$ matrix for this problem:

In [8]:
C = zeros(4,14); # understanding qestion: why is this a 4 x 14 array?

__Note__: Julia's Array syntax is similar to Matlab/Octave, except with square brackets. See [the Array documentation](https://docs.julialang.org/en/v1/base/arrays/) or various other [Julia tutorials on the web](https://www.tutorialspoint.com/julia/julia_arrays.htm) about working with the Array data structure.

### Task 3: Specify the problem object, and solve for the optimal configuration
Finally, build a `MySimpleBinaryVariableLinearChoiceProblem` model using the `build(...)` method, set this instance to the `problem` variable, and then pass the `problem` object to the `solve(...)` method. The `solve(...)` method will solve the `ILP` problem using the [GLPK.jl](https://github.com/jump-dev/GLPK.jl) interface to the [GLPK linear programming solver](https://www.gnu.org/software/glpk/).

In [9]:
α = configuration_array[:,1];
c = configuration_array[:,2];
I = 10000; # default budget is 10K USD

# Fill me in

#### Check: Are the choice constraints enforced?
We can only select a fixed number of items from each category. Does your solution reflect this restriction?

In [11]:
# fill me in

### Task 4: How does changing the $\alpha$-vector (or the budget $I$) influence the configuration choice?

In [10]:
# Fill me in - this is an open ended question, so mess around with the alpha - values and I and jot down some observations. 